# Fondamenti di elaborazione immagini

## Effettuiamo l'import delle librerie utilizzate nell'esercitazione.

In [ ]:
import cv2
import numpy as np
import matplotlib as mapli
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import ImageGrid
%matplotlib inline

Aggiungiamo alcune funzioni di utilita' per la scrittura del codice.

In [ ]:
def rgb(image : np.array) -> np.array:
    return cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

def gray(image : np.array) -> np.array:
    return cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

def grid(images : list[np.array], rows : int, cols : int, size : int, colors : list[str] = None) -> None:
    fig = plt.figure(figsize=(size,size))
    grid = ImageGrid(fig, 111, nrows_ncols=(rows, cols), axes_pad=0.1)

    if colors is not None:
        counter = 0
        for ax, im in zip(grid, images):
            ax.imshow(im, cmap=colors[counter])
            counter = (counter + 1) % len(colors)
        plt.show()
    else:
        for ax, im in zip(grid, images):
            ax.imshow(im)
        plt.show()

## _Le operazioni logiche, come quelle aritmetiche, agiscono pixel per pixel. La loro maggiore utilita' la si vede con immagini binarie, le maschere._

Di seguito un esempio di utilizzo delle tre operazioni piu' comuni: AND, OR, XOR.

Supponiamo di voler realizzare una rete che sappia leggere l'orario di un orologio a lancette. A disposizione abbiamo un dataset di immagini dove si vedono orologi con i loro quadranti a lancetti
e in diversi contesti. Per evitare che la rete si concentri a trovare features/caratteristiche su parti di immagini non di interesse, applichiamoci un maschera e oscuriamole.

In [ ]:
watches = [                                             # carichiamo immagini di 3 orologi a simulare un dataset.
    cv2.imread('./imgs/watches/watch_1.png'),
    cv2.imread('./imgs/watches/watch_2.png'),
    cv2.imread('./imgs/watches/watch_3.png')
]

grid([rgb(x) for x in watches], 1, 3, 30)

masks = [
    cv2.imread('./imgs/watches/masks/mask_1.png'),     # carichiamo le immagini delle maschere associate.
    cv2.imread('./imgs/watches/masks/mask_2.png'),
    cv2.imread('./imgs/watches/masks/mask_3.png')
]

grid([rgb(x) for x in masks], 1, 3, 30)

In [ ]:
masked_watches = []                                                 # ciclando immagini-maschere applichiamo un and logico.
for i in range(len(watches)):
    masked_watches.append(cv2.bitwise_and(watches[i], masks[i]))

grid([rgb(x) for x in masked_watches], 1, 3, 30)

Utilizzando _opencv_ ed il metodo _bitwise\_and_ si puo' realizzare un and logico, bit a bit, per ogni valore presente nell'immagine di input. Dal risultato si ottengono quindi solo i quadranti con le lancette e si va a rimuovere tutto cio' che non e' necessario.

* Rif: [bitwise_and](https://docs.opencv.org/4.x/d2/de8/group__core__array.html#ga60b4d04b251ba5eb1392c34425497e14)

Un ulteriore opzione, nella costruzione di un dataset, e' quella di combinare maschere di oggetti diversi al fine di ottenere una maschera composita. Anche in questo caso, _opencv_ ed il metodo _bitwise\_or_ permettono di raggiungere il risultato con un or logico.

* Rif: [bitwise_or](https://docs.opencv.org/4.x/d2/de8/group__core__array.html#gab85523db362a4e26ff0c703793a719b4)

In [ ]:
image = cv2.imread('./imgs/surf/image.png')
mask_board = cv2.imread('./imgs/surf/masks/board.png')
mask_surfer = cv2.imread('./imgs/surf/masks/surfer.png')

grid([rgb(image), rgb(mask_board), rgb(mask_surfer)], 1, 3, 30)

In [ ]:
mask_union = cv2.bitwise_or(mask_board, mask_surfer)

surfer_with_board = cv2.bitwise_and(image, mask_union)
board = cv2.bitwise_and(image, mask_board)
surfer = cv2.bitwise_and(image, mask_surfer)

grid([rgb(surfer_with_board), rgb(board), rgb(surfer)], 1, 3, 30)

Caso meno frequente e' quello nel quale si hanno a disposizione piu' maschere di oggetti potenzialmente sovrapposti o contenuti l'uno nell altro. Se si vuole ottenere una nuova maschera che presenti tutti i valori che non sono presenti in entrambe le maschere, possiamo utilizzare _opencv_ e il metodo _bitwise\_xor_.

* Rif: [bitwise_xor](https://docs.opencv.org/4.x/d2/de8/group__core__array.html#ga84b2d8188ce506593dcc3f8cd00e8e2c)

In [ ]:
watch = cv2.imread('./imgs/watches/watch_1.png')
mask_full_watch = cv2.imread('./imgs/watches/masks/mask_4.png')
mask_part_watch = cv2.imread('./imgs/watches/masks/mask_1.png')

grid([rgb(watch), rgb(mask_full_watch), rgb(mask_part_watch)], 1, 3, 30)

In [ ]:
mask_exclusive = cv2.bitwise_xor(mask_full_watch, mask_part_watch)

excluded = cv2.bitwise_and(watch, mask_exclusive)

grid([rgb(watch), rgb(excluded)], 1, 2, 20)

Per semplicita' si e' omessa l'operazione _bitwise\_not_ il cui risultato e' palese. Basti sapere che le operazioni sono comunque molteplici e consultabili direttamente dalla documentazione.

* Rif: [Operations on array](https://docs.opencv.org/4.x/d2/de8/group__core__array.html)